In [2]:
import numpy as np
import cv2

def region_selection(image):
    
     
    mask = np.zeros_like(image) 
    
    if len(image.shape) > 2:
        channel_count = image.shape[2]
        ignore_mask_color = (255,) * channel_count
    
    else:
        ignore_mask_color = 255
    
    rows, cols = image.shape[:2]
    bottom_left = [cols * 0.1, rows * 0.95]
    top_left    = [cols * 0.4, rows * 0.6]
    bottom_right = [cols * 0.9, rows * 0.95]
    top_right = [cols * 0.6, rows * 0.6]
    vertices = np.array([[bottom_left, top_left, top_right, bottom_right]], dtype=np.int32)
    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
  
    masked_image = cv2.bitwise_and(image, mask)
    return masked_image

def hough_transform(image):
    
    
    rho = 1            
    
    theta = np.pi/180
    
    threshold = 20    
    
    minLineLength = 20
   
    maxLineGap = 500    
    
    return cv2.HoughLinesP(image, rho = rho, theta = theta, threshold = threshold,
                        minLineLength = minLineLength, maxLineGap = maxLineGap)
    
def average_slope_intercept(lines):
   
    left_lines = [] 
    left_weights = [] 
    right_lines = [] 
    right_weights = [] 
    
    for line in lines:
        for x1, y1, x2, y2 in line:
            if x1 == x2:
                continue
            
            slope = (y2 - y1) / (x2 - x1)
            
            intercept = y1 - (slope * x1)
           
            length = np.sqrt(((y2 - y1) ** 2) + ((x2 - x1) ** 2))
        
            if slope < 0:
                left_lines.append((slope, intercept))
                left_weights.append((length))
            else:
                right_lines.append((slope, intercept))
                right_weights.append((length))
    # 
    left_lane = np.dot(left_weights, left_lines) / np.sum(left_weights) if len(left_weights) > 0 else None
    right_lane = np.dot(right_weights, right_lines) / np.sum(right_weights) if len(right_weights) > 0 else None
    return left_lane, right_lane

def pixel_points(y1, y2, line):
    
    if line is None:
        return None
    slope, intercept = line
    
    # Check if slope is not zero or infinity
    if slope == 0 or slope == float('inf') or slope == float('-inf'):
        return None
    
    x1 = int((y1 - intercept)/slope)
    x2 = int((y2 - intercept)/slope)
    y1 = int(y1)
    y2 = int(y2)
    return ((x1, y1), (x2, y2))

def lane_lines(image, lines):
    
    if lines is None:  
        return None, None
    
    left_lane, right_lane = average_slope_intercept(lines)
    y1 = image.shape[0]
    y2 = y1 * 0.6
    left_line = pixel_points(y1, y2, left_lane)
    right_line = pixel_points(y1, y2, right_lane)
    
  
    if left_line is not None and right_line is not None:
        return left_line, right_line
    else:
        return None, None

def draw_lane_lines(image, lines, color=[255, 0, 0], thickness=12):
    """
    Draw lines onto the input image.
        Parameters:
            image: The input test image (video frame in our case).
            lines: The output lines from Hough Transform.
            color (Default = red): Line color.
            thickness (Default = 12): Line thickness. 
    """
    line_image = np.zeros_like(image)
    for line in lines:
        if line is not None:
            cv2.line(line_image, *line, color, thickness)
    return cv2.addWeighted(image, 1.0, line_image, 1.0, 0.0)

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
   
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Apply Gaussian Blur
    blur = cv2.GaussianBlur(gray, (5, 5), 0)
    
    edges = cv2.Canny(blur, 50, 150)
    
    region = region_selection(edges)
    
    lines = hough_transform(region)
    
 
    left_lane, right_lane = lane_lines(frame, lines)
    result = draw_lane_lines(frame, (left_lane, right_lane))
    
    
    cv2.imshow('Lane Detection', result)
    
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the capture
cap.release()
cv2.destroyAllWindows()
